In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.preprocessing import Normalizer
from tensorflow.keras import utils
from sklearn.model_selection import train_test_split
import tensorflow as tf
#import tensorflow_federated as tff
from tensorflow import keras
from sklearn import preprocessing
from tensorflow.keras import Sequential
from tensorflow.keras.backend import set_floatx
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing import timeseries_dataset_from_array
from tensorflow.keras.layers import InputLayer, BatchNormalization, LSTM, Dropout, Dense, Bidirectional
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler
from keras import optimizers
#!pip install --upgrade nest_asyncio
#!pip install --upgrade tensorflow_federated

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
str_time = '10:15:41.0000001'

split = str_time.split(':')
new_value = float(split[0]) * 60 * 60
new_value += float(split[1]) * 60
new_value += float(split[2])

In [ ]:
new_value

In [ ]:
df1 = pd.read_csv('../input/smartilizer-dataset/Terra-D1-multi-labeled-interpolated.csv')
print('Before: ', df1['label'].size)
df1 = df1[np.all((df1.label.apply(float.is_integer), df1.label != 0), axis=0)]
print('After: ', df1['label'].size)

df2 = pd.read_csv('../input/commercial-vehicles-sensor-data-set/Terra-D2-multi-labeled-interpolated.csv')
print('Before: ', df2['label'].size)
df2 = df2[np.all((df2.label.apply(float.is_integer), df2.label != 0), axis=0)]
print('After: ', df2['label'].size)



print('Summary: ', df1['label'].size + df2['label'].size)

In [ ]:
df1 = df1[:300000]

In [ ]:
df2 = df2.drop(['time'], axis = 1)
df2['label'] = df2['label'].astype(int) - 1
df2.to_csv('smartilizer-data.csv', sep = ',', index = False)

In [ ]:
df2[:865000]

In [ ]:
df1 = df1.drop(['time'], axis = 1)
df2 = df2.drop(['time'], axis = 1)

df1['label'] = df1['label'].astype(int) - 1
df2['label'] = df2['label'].astype(int) - 1

In [ ]:
subset1_1 = df2[:570000]
subset1_2 = df2[570000:]

In [ ]:
subset1_1_train = subset1_1[:int(subset1_1.shape[0] * 0.8)]
subset1_1_test = subset1_1[int(subset1_1.shape[0] * 0.8):]


subset1_2_train = subset1_2[:int(subset1_2.shape[0] * 0.8)]
subset1_2_test = subset1_2[int(subset1_2.shape[0] * 0.8):]

set2_train = df1[:int(df1.shape[0] * 0.8)]
set2_test = df1[int(df1.shape[0] * 0.8):]

In [ ]:
set2_train

In [ ]:
all_test = pd.concat([subset1_1_test,subset1_2_test,set2_test], axis = 0)

In [ ]:
all_test

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt


t = subset1_1_train['time']
l = subset1_1_train['label']

t1 = subset1_1_test['time']
l1 = subset1_1_test['label']



t2 = subset1_2_train['time']
l2 = subset1_2_train['label']

t3 = subset1_2_test['time']
l3 = subset1_2_test['label']


t4 = set2_train['time']
l4 = set2_train['label']

t5 = set2_test['time']
l5 = set2_test['label']





In [ ]:
t5 = df1['time']
l5 = df1['label']

In [ ]:
plt.figure(figsize=(23, 5))
plt.title("Изменение состояний") 
plt.xlabel("Время") 
plt.ylabel("Состояние") 
plt.xlim(0, 17200)
plt.grid()      

plt.plot(t1, l1, 'r')
plt.plot(t2, l2, 'b')
plt.plot(t3, l3, 'r')


In [ ]:
plt.figure(figsize=(23, 5))
plt.title("Изменение состояний") 
plt.xlabel("Время") 
plt.ylabel("Состояние") 
plt.xlim(0, 6500)
plt.grid()      
plt.plot(t4, l4, 'b')
plt.plot(t5, l5, 'r')

In [ ]:
subset1_1_train

In [ ]:
#subset1_1_train.to_csv('subset1_1_train_6k.csv',sep = ',', index=True)
#subset1_1_test.to_csv('subset1_1_test_6k.csv',sep = ',', index=True)

#subset1_2_train.to_csv('subset1_2_train_6k.csv',sep = ',', index=True)
#subset1_2_test.to_csv('subset1_2_test_6k.csv',sep = ',', index=True)

set2_train.to_csv('set2_train_3k.csv',sep = ',', index=True)
set2_test.to_csv('set2_test_3k.csv',sep = ',', index=True)

In [ ]:
subset1_1_train

In [ ]:
big_data_train = df2[:int(df2.shape[0] * 0.8)]
big_data_test =  df2[int(df2.shape[0] * 0.8):]
small_data_train = df1[:int(df1.shape[0] * 0.8)]
small_data_test =  df1[int(df1.shape[0] * 0.8):]

In [ ]:
df2['time'].max() * 0.8

In [ ]:
e = big_data_test['label'].unique()

In [ ]:
e

In [ ]:
small_data_test.shape[0] + big_data_test.shape[0]

In [ ]:
big_data_test.to_csv('sw_1_big_test.csv',sep = ',', index=False)
big_data_train.to_csv('sw_1_big_train.csv',sep = ',', index=False)

small_data_test.to_csv('sw_2_small_test.csv',sep = ',', index=False)
small_data_train.to_csv('sw_2_small_train.csv',sep = ',', index=False)

In [ ]:
a = '5.0'
int(a)

# Classes

**1 -  idle;** the vehicle is not busy with the engine on.

**2 - driving;** the dumper is moving forward or backwards
(both hauling and driving empty).

**3 -  loading;** the dumper is being loaded with mass, generally
idle with the engine on. The time between each bucket is
usually around 25 seconds. Such in-between time is also
labeled as loading, i.e. not as idle.

**4 - dumping;** the vehicle is loosing its mass from the bed.
This activity starts when the dumper begins tilting its bed
and ends when the bed is back at its initial position.

**5 - engine-off;** the dumper is inactive with the engine off.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
"""
t = df1['time']
l = df1['label']
x = df1['wx']
y = df1['wy']
z = df1['wz']

t2 = df2['time']
l2 = df2['label']
x2 = df2['wx']
y2 = df2['wy']
z2 = df2['wz']
"""

t = df1['time']
l = df1['label']
x = df1['gFx']
y = df1['gFy']
z = df1['gFz']



t2 = df2['time']
l2 = df2['label']
x2 = df2['gFx']
y2 = df2['gFy']
z2 = df2['gFz']




In [ ]:
l2

In [ ]:
4500 * 0.8

In [ ]:
plt.figure(figsize=(23, 5))
plt.title("Изменение состояний") 
plt.xlabel("Время") 
plt.ylabel("Состояние") 
plt.xlim(0, 4000)
plt.grid()      
plt.plot(t, x)
plt.plot(t, y)
plt.plot(t, z)
plt.plot(t, l)

In [ ]:
plt.figure(figsize=(23, 5))
plt.title("Изменение состояний") 
plt.xlabel("Время") 
plt.ylabel("Состояние") 
plt.xlim(0, 17000)
plt.grid()      
plt.plot(t2, x2)
plt.plot(t2, y2)
plt.plot(t2, z2)
plt.plot(t2, l2)

In [ ]:
dataset = pd.concat([df1,df2],axis=0)
dataset = dataset.drop(['time'], axis = 1)

In [ ]:
grouped = dataset.label.unique()

In [ ]:
grouped.astype(int)

In [ ]:
for i in grouped:
    name = "label%d" % i
    print(type(name))
    globals()[name] = dataset[dataset['label'] == i]    

In [ ]:
def get_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

In [ ]:
labels = [label1,label2, label3, label4, label5]

for example in labels:
    percent = example.shape[0] /  2053015
    print(get_df_name(example), 'ratio is ', percent)

In [ ]:
def check_ratio(labels):
    grouped = labels.label.unique()
    grouped = grouped.astype(int)
    grouped.sort()
    for example in grouped:
        tmp = labels.loc[labels['label'] == example]
        ratio = tmp.shape[0] / labels.shape[0]
        print('Label ', example,' ratio is', ratio )
    print(labels.shape[0])
    
    

In [ ]:
def divide_data(df, ratio1 = 0.2, ratio2 = 0.3):
    
    Y = df['label']
    Y = Y.to_frame()
    X = df.drop(['label'], axis = 1)
    
    shape = X.shape[0]
    
    train_X1, x, train_Y1, y = train_test_split(X, Y, test_size=(1-ratio1), random_state=1)
    train_X2, x, train_Y2, y = train_test_split(x,y, test_size = 1 - (train_X1.shape[0] / x.shape[0]))
    
    train_X3, x, train_Y3, y = train_test_split(x,y, test_size = (1-ratio2), random_state=1)
    train_X4, test_x, train_Y4, test_y = train_test_split(x,y, test_size = 1 - (train_X3.shape[0] / x.shape[0]))

    
    print('train_1 is ', train_X1.shape[0] / shape * 100, '% of all data')
    print('train_2 is ', train_X2.shape[0] / shape * 100, '% of all data')
    print('train_3 is ', train_X3.shape[0] / shape * 100, '% of all data')
    print('train_4 is ', train_X4.shape[0] / shape * 100, '% of all data')
    print('test is ', test_x.shape[0] / shape * 100, '% of all data')
    
    
    return train_X1, train_Y1, train_X2,train_Y2,  train_X3,train_Y3, train_X4,train_Y4, test_x, test_y

In [ ]:
def process_labels(arr):
    print('***************')
    print('Check ratio for', get_name(arr))
    check_ratio(arr)
    print('***************')
    arr = arr['label'] - 1
    arr = utils.to_categorical(arr, 5)
    return arr

In [ ]:
train_X

In [ ]:
train_X1, train_Y1, train_X2,train_Y2,  train_X3,train_Y3, train_X4,train_Y4, test_x, test_y = divide_data(dataset, ratio1=0.1)

train_Y1 = process_labels(train_Y1)
train_Y2 = process_labels(train_Y2)
train_Y3 = process_labels(train_Y3)
train_Y4 = process_labels(train_Y4)
test_y = process_labels(test_y)



In [ ]:
train_X = np.array(train_X)

In [ ]:

train_X0 = subset1_1_train.drop(['label'], axis = 1)
train_Y0 = subset1_1_train['label']
train_Y0 = utils.to_categorical(train_Y0, 5)

test_X0 = subset1_1_test.drop(['label'], axis = 1)
test_Y0 = subset1_1_test['label']
test_Y0 = utils.to_categorical(test_Y0, 5)



train_X1 = subset1_2_train.drop(['label'], axis = 1)
train_Y1 = subset1_2_train['label']
train_Y1 = utils.to_categorical(train_Y1, 5)

test_X1 = subset1_2_test.drop(['label'], axis = 1)
test_Y1 = subset1_2_test['label']
test_Y1 = utils.to_categorical(test_Y1, 5)



train_X2 = set2_train.drop(['label'], axis = 1)
train_Y2 = set2_train['label']
train_Y2 = utils.to_categorical(train_Y2, 5)

test_X2 = set2_test.drop(['label'], axis = 1)
test_Y2 = set2_test['label']
test_Y2 = utils.to_categorical(test_Y2, 5)

In [ ]:
train_X0

In [ ]:
a = np.append(a, train_X1)

In [ ]:
x = get_name(train_X1)[-1]

In [ ]:
train_X1

In [ ]:
model_v = 'Dense'
def create_model(train_X, train_Y):
  model = tf.keras.Sequential([
      Dense(train_X.shape[1], input_shape=(train_X.shape[1],)),
      BatchNormalization(),
      Dense(256, activation='relu'),
      Dropout(.2),
      BatchNormalization(),
      Dense(64, activation='relu'),
      Dropout(.2),
      Dense(train_Y.shape[1], activation='softmax')
  ])
  print(train_X.shape[1], train_Y.shape[1])
  return model

In [ ]:
for i in range(3):
    exec('x = train_Y%s' % i)
    print(x.shape)
    

In [ ]:
for i in range(3):
    exec('train_X = train_X%s' % i)
    exec('test_X = test_X%s' % i)
    
    exec('train_Y = train_Y%s' % i)
    exec('test_Y = test_Y%s' % i)
    
    print('Training on set # ', i+1)
    
    print('Train data shape: ', train_X.shape)
    print('Test data shape: ', test_X.shape)
    


    model_1 = create_model(train_X, train_Y)
    opt = optimizers.SGD(learning_rate=0.01)
    _opt = optimizers.Adam(learning_rate=0.01)

    model_1.compile(optimizer=_opt, loss = 'categorical_crossentropy', metrics=['accuracy'])

    print('Fitting model...')
    #norm = Normalizer()
    #train_X1 = norm.fit_transform(train_X1)
    history = model_1.fit(train_X,train_Y, epochs=20, batch_size=32)


    score, acc = model_1.evaluate(test_X, test_Y, batch_size = 32)
    print('Loss: ', score, 'Acc: ', acc)


In [ ]:

model_test = create_model(train_X2, train_Y2)
opt = optimizers.SGD(learning_rate=0.01)
_opt = optimizers.Adam(learning_rate=0.01)

model_test.compile(optimizer=_opt, loss = 'categorical_crossentropy', metrics=['accuracy'])

print('Fitting model...')
#norm = Normalizer()
#train_X1 = norm.fit_transform(train_X1)
history = model_test.fit(train_X2,train_Y2, epochs=20, batch_size=32)

In [ ]:

score, acc = model_test.evaluate(test_X2, test_Y2, batch_size = 32)
print('Loss: ', score, 'Acc: ', acc)

In [ ]:
model_2 = create_model(train_X, train_Y)
opt = optimizers.SGD(learning_rate=0.01)
_opt = optimizers.Adam(learning_rate=0.01)

model_2.compile(optimizer=opt, loss = 'categorical_crossentropy', metrics=['accuracy'])

print('Fitting model...')
#norm = Normalizer()
#train_X1 = norm.fit_transform(train_X1)
history = model_2.fit(train_X0,train_Y0, epochs=20, batch_size=32)


score, acc = model_2.evaluate(test_X1, test_Y1, batch_size = 32)
print('Loss: ', score, 'Acc: ', acc)

In [ ]:
score, acc = model_2.evaluate(test_X0, test_Y0, batch_size = 32)
print('Loss: ', score, 'Acc: ', acc)

In [ ]:
plt.plot(history.history['accuracy'],
        label = 'train')
plt.plot(history.history['val_accuracy'],
        label = 'validation')
plt.xlabel('Epoch')
plt.ylabel('% success')
plt.legend()
plt.show()

In [ ]:
test_x = norm.fit_transform(test_x)

In [ ]:
test_x

In [ ]:
score, acc = model_1.evaluate(test_X, test_Y, batch_size = 32)
print('Loss: ', score, 'Acc: ', acc)

In [ ]:
test_Y.shape

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
t = set2_test['time']
x = set2_test['label']
plt.figure(figsize=(23, 5))
plt.title("Изменение состояний") 
plt.xlabel("Время") 
plt.ylabel("Состояние") 
plt.xlim(3000, 4000)
plt.grid()      
#plt.plot(t, x, 'ro')
plt.plot(t, x)

In [ ]:
pred = model_1.predict(test_X)

In [ ]:
x = np.array([])
y = np.array([])
for i in range(test_Y.shape[0]):
    x = np.append(x, np.argmax(test_Y[i]))
    y = np.append(y, np.argmax(pred[i]))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix


print(confusion_matrix(x, y))

print('Classification Report')
target_names = ['1', '2', '3','4','5']
print(classification_report(x, y, target_names=target_names))

In [ ]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier

# Defining and fitting a DecisionTreeClassifier instance
tree = DecisionTreeClassifier(max_depth = 19)
tree.fit(train_X, train_Y)

In [ ]:
from sklearn.tree import export_graphviz

# Creates dot file named tree.dot
export_graphviz(
            tree,
            out_file =  "myTreeName.dot",
            feature_names = list(train_X.columns),
            filled = True,
            rounded = True)

In [ ]:
pred = tree.predict(test_X)



In [ ]:
test_Y

In [ ]:
pred

In [ ]:
train_X.shape

In [ ]:
pred.shape


In [ ]:
x = np.array([])
y = np.array([])
for i in range(test_Y.shape[0]):
    x = np.append(x, np.argmax(test_Y[i]))
    y = np.append(y, np.argmax(pred[i]))
    

In [ ]:
x.shape

In [ ]:
t = np.array(t)

In [ ]:
t

In [ ]:
y

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
t = subset1_2_test['time']
plt.figure(figsize=(23, 5))
plt.title("Изменение состояний") 
plt.xlabel("Время") 
plt.ylabel("Состояние") 
plt.xlim(14000, 17000)
plt.grid()      
#plt.plot(t, x, 'ro')
plt.plot(t,y)

In [ ]:
plt.figure(figsize=(23, 5))
plt.title("Изменение состояний") 
plt.xlabel("Время") 
plt.ylabel("Состояние") 
plt.xlim(0, 17000)
plt.grid()      
plt.plot(t2, x2)
plt.plot(t2, y2)
plt.plot(t2, z2)
plt.plot(t2, l2)

In [ ]:
pred[1000]

In [ ]:
model_1.save('model1')

In [ ]:
0.11001388674587255 * 77772 

In [ ]:
model_2 = keras.models.load_model('model1')

In [ ]:
train_X3 = norm.fit_transform(train_X3)

In [ ]:
train_X3

In [ ]:
history2 = model_2.fit(train_X3, train_Y3, epochs=20, validation_split = 0.1,batch_size=32)

In [ ]:
plt.plot(history2.history['accuracy'],
        label = 'train')
plt.plot(history2.history['val_accuracy'],
        label = 'validation')
plt.xlabel('Epoch')
plt.ylabel('% success')
plt.legend()
plt.show()

In [ ]:
score, acc = model_2.evaluate(test_x, test_y, batch_size = 32)
print('Loss: ', score, 'Acc: ', acc)

In [ ]:
data  =  df2.drop(['time'], axis = 1)
df2[:1050000]

In [ ]:
number = 1050000

train_data = data[:number]
test_data = data[number:]

train_data = train_data.sample(number)

test_Y = test_data['label']
test_Y = test_Y.to_frame()
test_X = test_data.drop(['label'], axis = 1)

train_Y = train_data['label']
train_Y = train_Y.to_frame()
train_X = train_data.drop(['label'], axis = 1)

test_Y = process_labels(test_Y)
train_Y = process_labels(train_Y)

In [ ]:
model = create_model(train_X, train_Y)
opt = optimizers.SGD(learning_rate=0.01)

model.compile(optimizer=opt, loss = 'categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(train_X, train_Y, epochs=10, batch_size=32)

In [ ]:
score, acc = model.evaluate(test_X, test_Y, batch_size = 32)
print('Loss: ', score, 'Acc: ', acc)

In [ ]:
pred = model.predict(test_X)

# **lstm network**

In [ ]:
from tensorflow.keras.preprocessing import timeseries_dataset_from_array
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from scipy import stats
from sklearn.preprocessing import OneHotEncoder

In [ ]:
time_points = 600
BATCH_SIZE = 32

#Sweden
arg_num = 8
class_num = 5


In [ ]:
df1 = pd.read_csv('../input/smartilizer-dataset/Terra-D1-multi-labeled-interpolated.csv')
print('Before: ', df1['label'].size)
df1 = df1[np.all((df1.label.apply(float.is_integer), df1.label != 0), axis=0)]
print('After: ', df1['label'].size)

df2 = pd.read_csv('../input/commercial-vehicles-sensor-data-set/Terra-D2-multi-labeled-interpolated.csv')
print('Before: ', df2['label'].size)
df2 = df2[np.all((df2.label.apply(float.is_integer), df2.label != 0), axis=0)]
print('After: ', df2['label'].size)



print('Summary: ', df1['label'].size + df2['label'].size)

In [ ]:
df2['label'] = df2['label'].astype(int) - 1
subset1_1 = df2[:570000]
subset1_2 = df2[570000:]

In [ ]:

subset1_2_train = subset1_2[:int(subset1_2.shape[0]*0.8)]
subset1_2_test = subset1_2[int(subset1_2.shape[0]*0.8):]

In [ ]:
subset1_2_train

In [ ]:

def create_dataset(X, y, time_steps=1, step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - time_steps, step):
        v = X.iloc[i:(i + time_steps)].values
        labels = y.iloc[i: i + time_steps]
        Xs.append(v)
        ys.append(stats.mode(labels)[0][0])
    return np.array(Xs), np.array(ys).reshape(-1, 1)

In [ ]:
TIME_STEPS = 300
STEP = 100

train_X, train_Y = create_dataset(
    subset1_2_train[['gFx', 'gFy', 'gFz','wx', 'wy', 'wz', 'speed']],
    subset1_2_train.label,
    TIME_STEPS,
    STEP
)

test_X, test_Y = create_dataset(
    subset1_2_test[['gFx', 'gFy', 'gFz','wx', 'wy', 'wz', 'speed']],
    subset1_2_test.label,
    TIME_STEPS,
    STEP
)

In [ ]:
train_X.shape

In [ ]:
train_X.shape

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

enc = enc.fit(train_Y)

train_Y = enc.transform(train_Y)
test_Y = enc.transform(test_Y)

In [ ]:
train_Y.max()

In [ ]:
enc.categories_[0]

In [ ]:
opt = optimizers.SGD(learning_rate=0.01)

model_v = '0.1' #CONST
def create_model():
  model = Sequential([
    Bidirectional(
    LSTM(128, activation='tanh',input_shape=[train_X.shape[1], train_X.shape[2]], dropout=0.5)),
    Dense(128, activation='relu'),
    Dense(train_Y.shape[1], activation='softmax')
  ])

  model.compile(
    loss='categorical_crossentropy',
    optimizer=opt,
    metrics=['acc'],
  )
  return model

In [ ]:
model_test = create_model()

In [ ]:
history = model_test.fit(
    train_X, train_Y,
    epochs=20,
    batch_size=32,
    shuffle=False
)

In [ ]:
model_test.evaluate(test_X, test_Y)

In [ ]:
a = np.array([[[-34.1320000e-01, -2.0000000e-04,  9.5040000e-01, -6.2600000e-02,
       -2.1800000e-02,  1.5560000e-01,  25.9824322e-01]]])

In [ ]:
y_pred = model_test.predict(test_X)

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

def plot_cm(y_true, y_pred, class_names):
  cm = confusion_matrix(y_true, y_pred)
  fig, ax = plt.subplots(figsize=(18, 16)) 
  ax = sns.heatmap(
      cm, 
      annot=True, 
      fmt="d", 
      cmap=sns.diverging_palette(220, 20, n=7),
      ax=ax
  )

  plt.ylabel('Actual')
  plt.xlabel('Predicted')
  ax.set_xticklabels(class_names)
  ax.set_yticklabels(class_names)
  b, t = plt.ylim() # discover the values for bottom and top
  b += 0.5 # Add 0.5 to the bottom
  t -= 0.5 # Subtract 0.5 from the top
  plt.ylim(b, t) # update the ylim(bottom, top) values
  plt.show() # ta-da!

In [ ]:
enc.categories_[0]

In [ ]:
a = np.array([0, 1, 2, 3])

In [ ]:
a

In [ ]:
a = np.array([])
for i in test_Y:
    a = np.append(a,np.argmax(i))

In [ ]:
np.unique(a)

In [ ]:
plot_cm(
  enc.inverse_transform(test_Y),
  enc.inverse_transform(y_pred),
  a
)